In [1]:
! pip install transformers
! pip install datasets

In [2]:
cd '/content/drive/MyDrive/CAMemBERT2'

/content/drive/MyDrive/CAMemBERT2


In [3]:
from transformers import TrainingArguments,Trainer,AutoModelForSequenceClassification,AutoModel
import numpy as np
from math import floor
import torch
from torch.cuda import is_available
from collections import defaultdict
from scipy.stats import spearmanr
from sklearn.metrics import cohen_kappa_score,classification_report,fbeta_score
from models_f import MultiTaskModel
from trainer_f import MultiTaskModelTrainer
from trainer_utils import (training_args,training_kwargs,get_dataset,
                           freeze_layers,normalise_scores,create_params_grid)

In [4]:
pretrained_model_name = 'distilroberta-base'
model_type = 'roberta' if 'roberta' in pretrained_model_name else 'bert'
tasks = ['aes','ged']
dataset_obj,dataset_dict = get_dataset(tasks,pretrained_model=pretrained_model_name,max_length=512)
dataset_dict = dataset_dict.map(normalise_scores)

/content/drive/My Drive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/My Drive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/My Drive/CAMemBERT2/preprocessing_f.py:53: ParserWarning:

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ex/s]

  0%|          | 0/1057 [00:00<?, ?ex/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ex/s]

In [5]:
params = {}

In [6]:
trainer_kwargs,model_kwargs = training_kwargs(
    tasks=tasks,
    optimizer_weighting='dynamic',
    class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
    trainer=True,
    early_stopping_patience=None,
    shared_encoder_n_layers=2,
    output_layer={'aes': -1, 'ged': 0, 'ner': 0},
    normalised_values=[40,0],
    save_results=True)

args = training_args(lr=5e-4,epochs=4)


In [7]:
model = MultiTaskModel(kwargs_dict = model_kwargs,pretrained_model=pretrained_model_name)

model = freeze_layers(model,frozen_layers='all',model_type=model_type)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weig

In [ ]:
trainer = MultiTaskModelTrainer(
    model,
    args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['dev'],
)
trainer.kwargs_dict = trainer_kwargs
trainer.test_dataset = dataset_dict['test']
trainer.train()

***** Running training *****
  Num examples = 1057
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 532


Step,Training Loss
500,0.357300


aes_weight_coef : 1.0000019475681077e-12
ged_weight_coef : 1.0

losses {'aes': 4.4772571127508733e-13, 'ged': 0.6033476193745931, 'total_loss': 0.6033476193750409}

aes_metrics {'rmse_aes': 25.416032018084646, 'pearson_aes': -0.09892063483780555, 'spearman_aes': -0.12250934248127474, 'kappa_aes': -0.0013264554163596731}

ged               precision    recall  f1-score   support

           0       0.95      0.64      0.77     27581
           1       0.26      0.79      0.39      4305

    accuracy                           0.66     31886
   macro avg       0.60      0.72      0.58     31886
weighted avg       0.86      0.66      0.72     31886

ged_metrics {'accuracy_ged': 0.6632377846076648, 'f1_score_macro_ged': 0.5779769234963393, 'f1_score_weighted_ged': 0.7164455114971218, 'f_0_5_ged': 0.5592569496865667}

aes_weight_coef : 9.999999974752427e-07
ged_weight_coef : 0.9999989867210388

losses {'aes': 2.3509865313779226e-08, 'ged': 0.5908691353268094, 'total_loss': 0.5908691588366746



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=532, training_loss=0.3366277991492945, metrics={'train_runtime': 91.3221, 'train_samples_per_second': 46.298, 'train_steps_per_second': 5.815, 'total_flos': 0.0, 'train_loss': 0.3366277991492945, 'epoch': 4.0})